In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_csv('./data/ybigta_sdss_train.csv',index_col='id')
train = train.reset_index()

In [4]:
from sklearn.model_selection import train_test_split
X = train.drop(['type'], axis = 1)
y = train['type']

In [ ]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
lgbm_model = LGBMClassifier(boosting_type='gbdt', objective='multiclass',n_estimators=1750, bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)


param_dist = {'max_depth': [15,30,45],
              'learning_rate': [0.001,0.005,0.01],
              "num_leaves": [80, 120, 160],
             }
grid_search = GridSearchCV(lgbm_model, n_jobs=12, param_grid=param_dist, cv=5, scoring="neg_log_loss", verbose=5)
grid_search.fit(X,y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.


In [ ]:
print('GridSearch 최적 parameter: {}'.format(grid_search.best_params_),
      'GridSearch 최고 Validation Score: {:.3f}'.format(grid_search.best_score_), sep = '\n')

In [8]:
# from sklearn.preprocessing import RobustScaler
# rb_scaler = RobustScaler()
# X_for_scaling = X.iloc[:,1:]
# # type과 id를 제외하고 학습
# rb_scaler.fit(X_for_scaling)
# # 학습후 변환
# train_rbscaled = rb_scaler.transform(X_for_scaling)
# train_rbscaled
# # 학습후 변환한 데이터를 다시 원래 데이터로 만들기
# X_rbscaled = pd.DataFrame(train_rbscaled, columns = X_for_scaling.columns)
# X_rbscaled = pd.concat([X[['fiberID']], X_rbscaled], axis=1)
# X_rbscaled.head()

# data_scaled = pd.concat([data[['id',"type"]], X_rbscaled], axis=1)
# data_scaled.head()

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,415567,QSO,106,-0.435893,-0.291574,-0.061425,-0.111254,0.045898,-0.526436,-0.469058,...,-0.878988,-0.808048,-0.702073,-0.717412,-0.654057,-1.076404,-0.950766,-0.802411,-0.819421,-0.727297
1,733874,QSO,492,-0.580244,-0.413752,-0.220812,-0.112515,0.093583,-0.599909,-0.368806,...,-0.453638,-0.124812,0.072741,0.148958,0.295791,-0.463942,-0.125875,0.075270,0.155184,0.285476
2,1009150,QSO,388,-0.185463,-0.377135,-0.229722,-0.127228,0.040244,-0.152645,-0.314845,...,0.026496,-0.095064,0.083110,0.147472,0.251690,0.011104,-0.094863,0.073686,0.147607,0.248041
3,803041,QSO,531,0.616269,0.437863,0.607836,0.845503,0.851924,0.742433,0.546231,...,0.794639,0.615596,0.795158,0.924695,0.971942,0.968832,0.616370,0.785311,0.919010,0.899853
4,432241,QSO,180,0.928778,0.372564,0.167249,0.202501,0.223579,1.037113,0.462229,...,1.153932,0.575626,0.408968,0.413842,0.363370,1.370472,0.562681,0.404408,0.398004,0.386068


In [ ]:
orb_type = [
    "GALAXY",
    "QSO",
    "REDDEN_STD",
    "ROSAT_D",
    "SERENDIPITY_BLUE",
    "SERENDIPITY_DISTANT",
    "SERENDIPITY_FIRST",
    "SERENDIPITY_MANUAL",
    "SERENDIPITY_RED",
    "SKY",
    "SPECTROPHOTO_STD",
    "STAR_BHB",
    "STAR_BROWN_DWARF",
    "STAR_CARBON",
    "STAR_CATY_VAR",
    "STAR_PN",
    "STAR_RED_DWARF",
    "STAR_SUB_DWARF",
    "STAR_WHITE_DWARF"
]
def sampling(df, orb_type): #df : 데이터프레임, orb_type : 천체타입 리스트
    copy_train = df.copy()
    for t in orb_type:
        print("현재 type : ", t)
        insert_row = int(input("랜덤샘플링으로 추가할 개수를 입력하세요"))
        temp = df[df["type"]==t].sample(n = insert_row, replace=True)  #t는 tpye의 종류, replace = True 옵션을 줘야 개수가 더 많아도 샘플링 가능
        copy_train = pd.concat([copy_train, temp], axis = 0) #row로 붙여넣기
    return copy_train

In [ ]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer

# 모델 기본 틀 생성
lgbm_model = LGBMClassifier(
    objective='binary', num_leaves=128, feature_fraction=0.9, reg_lambda=0.2, silent=False)

# 그리드 서치 인자 조합 생성
param_dist = { "boosting_type": ['goss'],
              "max_depth" : [15, 30, 45],
              "learning_rate" : [0.01 , 0.05, 0.1],
              "n_estimators" : [1500, 2000, 2500],
            }

# 점수를 위한 로그로스 객체 생성
# LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

# lgbm_model = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=10,
#                                 learning_rate=0.1, n_estimators=2000, max_depth=15,
#                                 bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)

In [6]:
grid_search = GridSearchCV(lgbm_model, n_jobs=-1, param_grid = param_dist, cv=3, scoring="neg_log_loss")

In [ ]:
grid_search.fit(X_rbscaled, y)
grid_search.best_estimator_
grid_search.best_score_

In [ ]:
grid_search.best_estimator_
grid_search.best_score_